# Implementing a simple GAN using keras


# Importing the libs 

In [25]:
from tensorflow.keras.layers import LeakyReLU, Dense, Input
import tensorflow as tf
from tensorflow.keras.models import Model

# defining the generator and discriminant model 

In [28]:
#define the generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(Dense(128, input_dim=100))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(784,activation='tanh'))
    return model

#define the discriminant model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(Dense(128, input_shape=(784,)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Building the generator

In [29]:
#build the generator
generator = build_generator()

/home/shobhit/.local/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/shobhit/.local/lib/python3.9/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


# Building and compile the discriminator

In [30]:
# build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])


In [ ]:
# Create the GAN by combining the generator and discriminator
discriminator.trainiable = False
gan_input = Input(shape=(100,))
generated_images = generator(gan_input)
gan_output = discriminator(generated_images)
gan = Model(gan_input, gan_output)


In [32]:
#compile the GAN
gan.compile(optimizer='adam', loss='binary_crossentropy')

# training the GAN

In [34]:
def train_gan(gan, generator, discriminator, X_train, epochs = 400, batch_size=128):

    #loop through epochs
    for epoch in range(epochs):
        #generate random noise as input for the generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)

        #get a random set of real images
        idz = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[idz]

        #label for the real and fake images
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))


        #train the discriminator on real and fake images separately
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(generates_images, fake_labels)

        #calculate the average loss fot the disciminator
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        #generate new noise and train the generator through the GAN model (note
        #we train the generator via the GAN model, where the discriminator
        #weights are frozen)
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, real_labels, return_dict=True)

        #print the progress every 10 epochs
        if epoch % 10 == 0:
            print(f"Epoch {epoch} - Discriminator Loss: {d_loss[0]}",
                  f"Generator loss: {g_loss['loss']}")

        return d_loss, g_loss